In [ ]:
# <-- Import libraries, custom functions, and load configuration & datasets <--

import yaml
import pandas as pd
import numpy as np
import datetime as dt
import re

# <-- Imports custom preprocessing functions from 'functions.py' <--

from functions import (drop_duplicates,
                       concat_dataframes,
                       remove_all_punctuation,
                       drop_irrelevant_columns,
                       standardize_column_names,
                       filter_by_regex_pattern,
                       standardize_dates
                       )

# <-- Loads YAML configuration to dynamically reference CSV output files. <--

config = None  # <-- Initialize config
try:
    with open("../config.yaml", "r") as file:
        config = yaml.safe_load(file)
except:
    print("Yaml configuration file not found!")
config

job_nyc1 = pd.read_csv(config['input_data']['file1'])
job_nyc2 = pd.read_csv(config['input_data']['file2'])

In [2]:
job_nyc1.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5069 entries, 0 to 5068
Data columns (total 30 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Job ID                         5069 non-null   int64  
 1   Agency                         5069 non-null   object 
 2   Posting Type                   5069 non-null   object 
 3   # Of Positions                 5069 non-null   int64  
 4   Business Title                 5069 non-null   object 
 5   Civil Service Title            5069 non-null   object 
 6   Title Classification           5069 non-null   object 
 7   Title Code No                  5069 non-null   object 
 8   Level                          5069 non-null   object 
 9   Job Category                   5069 non-null   object 
 10  Full-Time/Part-Time indicator  4990 non-null   object 
 11  Career Level                   5069 non-null   object 
 12  Salary Range From              5069 non-null   f

In [3]:
job_nyc1["Preferred Skills"].isnull().sum()

np.int64(1375)

In [4]:
job_nyc2["Preferred Skills"].isnull().sum()

np.int64(2699)

In [5]:
job_nyc1["Preferred Skills"].value_counts(dropna=False)

Preferred Skills
NaN                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

In [6]:
display(pd.DataFrame({
    "non_nulls": job_nyc1.count(),
    "nulls": job_nyc1.isnull().sum(),
    "total": len(job_nyc1)
}))

,non_nulls,nulls,total
Job ID,5069,0,5069
Agency,5069,0,5069
Posting Type,5069,0,5069
# Of Positions,5069,0,5069
Business Title,5069,0,5069
Civil Service Title,5069,0,5069
Title Classification,5069,0,5069
Title Code No,5069,0,5069
Level,5069,0,5069
Job Category,5069,0,5069


In [7]:
display(pd.DataFrame({
    "non_nulls": job_nyc2.count(),
    "nulls": job_nyc2.isnull().sum(),
    "total": len(job_nyc2)
}))

,non_nulls,nulls,total
Job ID,6030,0,6030
Agency,6030,0,6030
Posting Type,6030,0,6030
# Of Positions,6030,0,6030
Business Title,6030,0,6030
Civil Service Title,6030,0,6030
Title Classification,6030,0,6030
Title Code No,6030,0,6030
Level,6030,0,6030
Job Category,6030,0,6030


In [8]:
df_merged = concat_dataframes(job_nyc1,job_nyc2)

In [9]:
df_merged = standardize_column_names(df_merged) 

In [10]:
display(df_merged.columns)

Index(['job_id', 'agency', 'posting_type', '#_of_positions', 'business_title',
       'civil_service_title', 'title_classification', 'title_code_no', 'level',
       'job_category', 'full-time/part-time_indicator', 'career_level',
       'salary_range_from', 'salary_range_to', 'salary_frequency',
       'work_location', 'division/work_unit', 'job_description',
       'minimum_qual_requirements', 'preferred_skills',
       'additional_information', 'to_apply', 'hours/shift', 'work_location_1',
       'recruitment_contact', 'residency_requirement', 'posting_date',
       'post_until', 'posting_updated', 'process_date'],
      dtype='object')

In [11]:
df_merged = drop_duplicates(df_merged,'job_id')

In [12]:
columns = ['job_id', 'posting_type', 'civil_service_title', 'title_classification', 
'title_code_no', 'full-time/part-time_indicator', 'work_location', 'division/work_unit',
'job_description', 'minimum_qual_requirements', 'additional_information', 'to_apply', 
'hours/shift', 'work_location_1', 'recruitment_contact', 'residency_requirement', 'posting_updated',
'process_date']

df_merged = drop_irrelevant_columns(df_merged, columns)

In [13]:
display(df_merged.columns)

Index(['agency', '#_of_positions', 'business_title', 'level', 'job_category',
       'career_level', 'salary_range_from', 'salary_range_to',
       'salary_frequency', 'preferred_skills', 'posting_date', 'post_until'],
      dtype='object')

In [14]:
display(df_merged.isnull().sum())

agency                  0
#_of_positions          0
business_title          0
level                   0
job_category            0
career_level            0
salary_range_from       0
salary_range_to         0
salary_frequency        0
preferred_skills     1933
posting_date            0
post_until           3270
dtype: int64

In [15]:
display(pd.DataFrame({
    "non_nulls": df_merged.count(),
    "nulls": df_merged.isnull().sum(),
    "total": len(df_merged)
}))

,non_nulls,nulls,total
agency,5271,0,5271
#_of_positions,5271,0,5271
business_title,5271,0,5271
level,5271,0,5271
job_category,5271,0,5271
career_level,5271,0,5271
salary_range_from,5271,0,5271
salary_range_to,5271,0,5271
salary_frequency,5271,0,5271
preferred_skills,3338,1933,5271


In [16]:
cat_cols = list(df_merged.select_dtypes(include=['object']))
cat_cols

['agency',
 'business_title',
 'level',
 'job_category',
 'career_level',
 'salary_frequency',
 'preferred_skills',
 'posting_date',
 'post_until']

In [17]:
# Clean up categorical columns (type = 'object') except dates
cat_cols = ['agency',
 'business_title',
 'level',
 'job_category',
 'career_level',
 'salary_frequency',
 'preferred_skills']
 
df_merged = remove_all_punctuation(df_merged,cat_cols)

In [18]:
df_merged[['preferred_skills','job_category','career_level']]

,preferred_skills,job_category,career_level
0,must have a valid pe license,engineering architecture planning,experienced nonmanager
1,student must currently be enrolled at a colleg...,engineering architecture planning,student
2,NaN,communications intergovernmental affairs techn...,experienced nonmanager
3,a baccalaureate degree from an accredited coll...,engineering architecture planning,experienced nonmanager
4,preference will be given to candidates with at...,engineering architecture planning,manager
...,...,...,...
10906,NaN,administration human resources,experienced nonmanager
10957,NaN,engineering architecture planning public safet...,experienced nonmanager
10961,NaN,communications intergovernmental affairs,manager
11037,NaN,communications intergovernmental affairs,manager


In [19]:
df_merged[['posting_date', 'post_until']]

,posting_date,post_until
0,06/08/2022,NaN
1,04/02/2024,11-JUN-2024
2,02/27/2024,26-FEB-2025
3,03/05/2024,29-MAY-2024
4,11/06/2023,NaN
...,...,...
10906,10/18/2024,NaN
10957,04/25/2025,NaN
10961,05/29/2025,NaN
11037,05/12/2025,NaN


In [20]:
date_cols = ['posting_date', 'post_until']
df_merged = standardize_dates(df_merged,date_cols)

d:\vscode101\first_project\notebooks\functions.py:51: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], errors='coerce', dayfirst=True)


In [21]:
df_merged[['posting_date', 'post_until']]

,posting_date,post_until
0,2022-08-06,NaT
1,2024-02-04,2024-06-11
2,NaT,2025-02-26
3,2024-05-03,2024-05-29
4,2023-06-11,NaT
...,...,...
10906,NaT,NaT
10957,NaT,NaT
10961,NaT,NaT
11037,2025-12-05,NaT


In [22]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5271 entries, 0 to 11047
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   agency             5271 non-null   object        
 1   #_of_positions     5271 non-null   int64         
 2   business_title     5271 non-null   object        
 3   level              5271 non-null   object        
 4   job_category       5271 non-null   object        
 5   career_level       5271 non-null   object        
 6   salary_range_from  5271 non-null   float64       
 7   salary_range_to    5271 non-null   float64       
 8   salary_frequency   5271 non-null   object        
 9   preferred_skills   3338 non-null   object        
 10  posting_date       2045 non-null   datetime64[ns]
 11  post_until         2001 non-null   datetime64[ns]
dtypes: datetime64[ns](2), float64(2), int64(1), object(7)
memory usage: 535.3+ KB


In [23]:
display(df_merged['business_title'])

0                                        region supervisor
1        2024bws010early warning remote modeling intern...
2                                     intelligence analyst
3                      senior coordinator capital projects
4                                                 director
                               ...                        
10906                        deputy director of onboarding
10957                             environmental specialist
10961                       deputy communications director
11037                  director for digital communications
11047     greenways project lead office of livable streets
Name: business_title, Length: 5271, dtype: object

In [24]:
regex_pattern = r"\b(data analyst|data engineer|data analyst)\w*\b"
df_data_analyst = filter_by_regex_pattern(df_merged,'business_title', regex_pattern)

d:\vscode101\first_project\notebooks\functions.py:44: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask = df[column].str.contains(regex_pattern, flags=re.IGNORECASE, na=False, regex=True)


In [25]:
df_data_analyst.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43 entries, 0 to 42
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   agency             43 non-null     object        
 1   #_of_positions     43 non-null     int64         
 2   business_title     43 non-null     object        
 3   level              43 non-null     object        
 4   job_category       43 non-null     object        
 5   career_level       43 non-null     object        
 6   salary_range_from  43 non-null     float64       
 7   salary_range_to    43 non-null     float64       
 8   salary_frequency   43 non-null     object        
 9   preferred_skills   20 non-null     object        
 10  posting_date       20 non-null     datetime64[ns]
 11  post_until         36 non-null     datetime64[ns]
dtypes: datetime64[ns](2), float64(2), int64(1), object(7)
memory usage: 4.2+ KB


In [26]:
display(pd.DataFrame({
    "non_nulls": df_data_analyst.count(),
    "nulls": df_data_analyst.isnull().sum(),
    "total": len(df_data_analyst)
}))

,non_nulls,nulls,total
agency,43,0,43
#_of_positions,43,0,43
business_title,43,0,43
level,43,0,43
job_category,43,0,43
career_level,43,0,43
salary_range_from,43,0,43
salary_range_to,43,0,43
salary_frequency,43,0,43
preferred_skills,20,23,43


In [27]:
regex_pattern = r"(sql|tableau|bi|phyton|eda|llm|ai|ml|pandas|numpy|agile)"
df_keywords = filter_by_regex_pattern(df_merged,'preferred_skills', regex_pattern)

d:\vscode101\first_project\notebooks\functions.py:44: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask = df[column].str.contains(regex_pattern, flags=re.IGNORECASE, na=False, regex=True)


In [28]:
df_keywords.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2648 entries, 0 to 2647
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   agency             2648 non-null   object        
 1   #_of_positions     2648 non-null   int64         
 2   business_title     2648 non-null   object        
 3   level              2648 non-null   object        
 4   job_category       2648 non-null   object        
 5   career_level       2648 non-null   object        
 6   salary_range_from  2648 non-null   float64       
 7   salary_range_to    2648 non-null   float64       
 8   salary_frequency   2648 non-null   object        
 9   preferred_skills   2648 non-null   object        
 10  posting_date       1025 non-null   datetime64[ns]
 11  post_until         813 non-null    datetime64[ns]
dtypes: datetime64[ns](2), float64(2), int64(1), object(7)
memory usage: 248.4+ KB


In [29]:
display(pd.DataFrame({
    "non_nulls": df_keywords.count(),
    "nulls": df_keywords.isnull().sum(),
    "total": len(df_keywords)
}))

,non_nulls,nulls,total
agency,2648,0,2648
#_of_positions,2648,0,2648
business_title,2648,0,2648
level,2648,0,2648
job_category,2648,0,2648
career_level,2648,0,2648
salary_range_from,2648,0,2648
salary_range_to,2648,0,2648
salary_frequency,2648,0,2648
preferred_skills,2648,0,2648


In [30]:
df_merged.to_csv(config['output_data']['file1'], index=False, sep=",", encoding="utf-8")
df_data_analyst.to_csv(config['output_data']['file2'], index=False, sep=",", encoding="utf-8")
df_keywords.to_csv(config['output_data']['file3'], index=False, sep=",", encoding="utf-8")